In [ ]:
from datasets import load_dataset

sentiment_dataset = load_dataset("sepidmnorozy/Korean_sentiment")
# label column name 변경 (-> labels)
sentiment_dataset = sentiment_dataset.rename_column("label", "labels")

In [ ]:
print(sentiment_dataset)

In [ ]:
# from sklearn.model_selection import train_test_split
#
# '''
# sklearn 의 train_test_split
# 데이터를 학습용 데이터와 테스트 용 데이터로 나누는 함수이다.
#
# train_test_split 의 반환값
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# X_train : 학습용 데이터 셋
# X_test : 테스트용 데이터의 셋
# y_train : 학습용 데이터의 label
# y_test : 테스트용 데이터의 label
# '''
#
# X_train_val, X_test, y_train_val, y_test = train_test_split(sentiment_input["train"]["text"],
#                                                             sentiment_input["train"]["label"], test_size=0.2,
#                                                             random_state=42)

In [ ]:
"""
토크나이저 불러오기
"""
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

In [ ]:
"""
토큰화 함수 정의
"""


def tokenize_funciton(sample):
    return tokenizer(
        sample["text"],  # 텍스트 데이터
        # padding=True,  # 패딩 적용
        padding="max_length",  # 최대 길이로 패딩
        truncation=True,  # 최대 길이를 넘어가는 데이터는 자름
        return_tensors="pt"  # 파이토치 텐서로 반환
    )

In [ ]:
"""
토크나이저 사용하기
"""

tokenized_sentiment_dataset = sentiment_dataset.map(
    tokenize_funciton,
    batched=True,  # 병렬 처리 활성화
    remove_columns=["text"])  # 원본 데이터에서 text 컬럼 제거 (메모리 최적화)

In [ ]:
"""
토크나이저가 적용된 데이터 셋 확인하기
"""
print(tokenized_sentiment_dataset)

In [ ]:
"""
데이터 셋 분리해서 준비하기
"""

# 학습용 데이터셋
train_dataset = tokenized_sentiment_dataset["train"]

# 검증용 데이터셋
validation_dataset = tokenized_sentiment_dataset["validation"]

# 테스트용 데이터셋
test_dataset = tokenized_sentiment_dataset["test"]

In [ ]:
"""
분리된 데이터 셋 확인하기
"""

# 학습용 데이터셋 확인
print(train_dataset[0])

# 검증용 데이터셋 확인
print(validation_dataset[0])

# 테스트용 데이터셋 확인
print(test_dataset[0])

In [ ]:
import torch
#
#
# class preprocess_dataset(torch.utils.data.Dataset):
#     def __init__(self, inputs, labels):
#         self.inputs = inputs
#         self.labels = labels
#
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
#         item["labels"] = torch.tensor(self.labels[idx])
#         return item
#
#     def __len__(self):
#         return len(self.labels)

In [ ]:
# sentiment_dataset = preprocess_dataset()

In [ ]:
# # senti = tokenizer(sentiment_dataset["train"][0])
# print(sentiment_dataset["train"][0])

In [ ]:
from transformers import TrainingArguments

# 하이처 파라미터 지정
training_args = TrainingArguments(
    output_dir="../results",  # 출력 디렉토리
    num_train_epochs=3,  # 학습할 에포크 수
    eval_strategy="steps",  # 평가 전략 : 에포크마다 검증 데이터셋에 대한 평가 지표 출력
    eval_steps=500,  # 500 스텝마다 평가
    per_device_train_batch_size=8,  # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,  # warmup 스텝 수
    weight_decay=0.01,  # 가중치 감쇠
    remove_unused_columns=False,  # 사용하지 않는 컬럼 제거
)

In [ ]:
# """
#
# """
# import evaluate
#
# # 정확도 메트릭을 불러온다.
# # 예측값과 실제 라벨을 비교하여 정확도를 계산한다.
# metric = evaluate.load("accuracy")

In [ ]:
# """
# 정확도 계산 함수 정의
# """
# import numpy as np
#
#
# def compute_metrics(eval_pred):
#     # logits : 모델의 출력값
#     # labels : 실제 라벨 값
#     logits, label = eval_pred
#
#     # 모델이 출력한 로짓에서 가장 높은 값을 가진 인덱스를 선택해서 예측 클래스를 만든다.
#     predictions = np.argmax(logits, axis=-1)
#
#     # predictions 와 실제 라벨을 비교하여 정확도를 계산한다.
#     # 정확도를 반환한다.
#     return metric.compute(predictions=predictions, references=label)

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": float(
            accuracy_score(labels, predictions)
        )
    }

In [ ]:
"""
모델 불러오기
"""

from transformers import AutoModel

model = AutoModel.from_pretrained("monologg/kobert")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
"""
BERT 분류기 사용하기
"""

from transformers import BertForSequenceClassification

clf_model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)

In [ ]:
"""
분류기 학습시키기
"""

from transformers import TrainingArguments

# 하이처 파라미터 지정
clf_training_args = TrainingArguments(
    output_dir="../clf_results",  # 출력 디렉토리
    num_train_epochs=3,  # 학습할 에포크 수
    eval_strategy="steps",  # 평가 전략 : 에포크마다 검증 데이터셋에 대한 평가 지표 출력
    eval_steps=500,  # 500 스텝마다 평가
    per_device_train_batch_size=8,  # 학습에 사용할 배치
    # 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,  # warmup 스텝 수
    weight_decay=0.01,  # 가중치 감쇠
    remove_unused_columns=False,  # 사용하지 않는 컬럼 제거
)

In [ ]:
"""
clf Trainer 불러오기
"""
from transformers import Trainer

clf_trainer = Trainer(
    model=clf_model,
    args=clf_training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
"""
mac gpu 사용하기
"""

import torch

print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}")

device = torch.device("mps")

clf_model.to(device)

In [ ]:
"""
clf Trainer 학습시키기
"""

clf_trainer.train()
